# Leitura de dados binários - um exemplo de otimização em Python
> As atividades de monitoramento do espectro de radiofrequência na ANATEL, especificamente aquelas que utilizam estações remotas de monitoramento, geram centenas de GB de dados. Um dos primeiros desafios para a utilização mais eficiente desses dados é a sua leitura eficiente. Nesse artigo é detalhada as iterações realizadas para resolver esse problema, primeiramente tornar essa leitura viável e posteriormente os passos de otimização aplicados no algoritmo para decodificar os dados de arquivos comprimidos 

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [spectrum monitoring, profiling, numpy, cython]
- image: images/multiplication.jpg
- hide_binder_badge: true
- author: Ronaldo S.A. Batista

![](images/knuth.jpeg "Crédito: https://twitter.com/lpolovets/status/816117631572807680")

O objeto desse artigo são arquivos binários com extensão `.bin` - gerados pela aplicação `Logger` embarcada nas estações de monitoramento do tipo [Rfeye Node 20-6](http://agc.com.br/produto/rfeye-node/).

Estes arquivos armazenam diversos metadados sobre a medição e blocos com informação numérica, que são as medidas em si, chamadas aqui de _dados de espectro_ ou _dados espectrais_. 

A versão recente da aplicação `Logger` gera dados comprimidos de maneira muito eficiente, o que torna a descompressão desafiadora para arquivos com muitos dados e bastante demorada caso seja feita de maneira "ingênua".

Mesmo que essa o problema apresentado aqui pareça obscuro, as técnicas de otimização podem ser aplicadas em outros contextos que sejam relevantes para quem lê.

> Note: Nos parágrafos a seguir irei passar por cima deliberadamente ( por não serem relevantes para a otimização e por pouco conhecimento no assunto ) de explicações sobre a parte de física / engenharia de telecomunicações e irei focar mais no problema específico de decodificação dos arquivos.

## Explorando o arquivo `.bin`

Cada arquivo `.bin` possui dados distintos em um mesmo arquivo, em dois níveis `blocos` e `thread_id`.

* Um `bloco` determina o tipo de dado: espectro, gps, dados textuais etc...
* O `thread_id` nada mais é que um identificador da faixa específica de varredura armazenada naquele bloco.

Com alguns exemplos fica muito mais fácil.

A função a seguir encapsula a leitura do arquivo e seus metadados e retorna um dicionário cujas chaves são as diferentes combinações de `blocos` e `thread_id` e os valores são as listas com os blocos. 

Cada bloco é uma classe python contendo seus atributos.Os detalhes de implementação dessa função podem ser ignorados.

> Tip: A biblioteca `rfpy` criada para o processamento desses arquivos faz amplo uso da biblioteca [fastcore](https://fastcore.fast.ai/). Esta expande as funcionalidades da linguagem python inspirada em atributos muito úteis de outras linguagens. Recomendo fortemente para quem deseja expandir o seu inventário de ferramentas python

In [ ]:
#hide
%load_ext autoreload
%load_ext line_profiler
%load_ext cython
%autoreload 2 

In [ ]:
import sys, os
from tqdm.notebook import tqdm
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
from multiprocessing import set_start_method, Pool
try:
    set_start_method("spawn")
except RuntimeError:
    pass
import gc
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from rfpy.parser import parse_bin
from rfpy.utils import public_attrs
from fastcore.foundation import L
import numpy as np
import pandas as pd
from pprint import pprint as pp
from IPython.display import display
import gc

In [ ]:
blocks = parse_bin('binfiles/rfeye002092_210223_T163131_MaskBroken.bin')
blocks = blocks['blocks']

for k,v in blocks.items():
    print(f'Tipo de Bloco: {k[0]}, Thread_ID: {k[1]}, Nº de blocos {len(v)}, Exemplo de Blocos: {v[:2]}')

Tipo de Bloco: 21, Thread_ID: 0, Nº de blocos 1, Exemplo de Blocos: [<rfpy.blocks.DType21 object at 0x000001FEE0073DC8>]
Tipo de Bloco: 42, Thread_ID: 0, Nº de blocos 2, Exemplo de Blocos: [<rfpy.blocks.DType42 object at 0x000001FEDDB88348>, <rfpy.blocks.DType42 object at 0x000001FEDDB88648>]
Tipo de Bloco: 42, Thread_ID: 301, Nº de blocos 1, Exemplo de Blocos: [<rfpy.blocks.DType42 object at 0x000001FEE0100448>]
Tipo de Bloco: 42, Thread_ID: 311, Nº de blocos 1, Exemplo de Blocos: [<rfpy.blocks.DType42 object at 0x000001FEE0100848>]
Tipo de Bloco: 42, Thread_ID: 321, Nº de blocos 1, Exemplo de Blocos: [<rfpy.blocks.DType42 object at 0x000001FEE0100A48>]
Tipo de Bloco: 42, Thread_ID: 331, Nº de blocos 1, Exemplo de Blocos: [<rfpy.blocks.DType42 object at 0x000001FEE0100BC8>]
Tipo de Bloco: 68, Thread_ID: 331, Nº de blocos 734, Exemplo de Blocos: [<rfpy.blocks.DType68 object at 0x000001FEE0100D08>, <rfpy.blocks.DType68 object at 0x000001FEDF14F388>]
Tipo de Bloco: 42, Thread_ID: 341, Nº

Vemos que esse arquivo tem diferentes tipos de blocos, o que nos interessa aqui são os blocos de espectro, blocos do tipo 68. 
Temos diferentes blocos do tipo 68, vamos observar o que os diferencia:

In [ ]:
b301 = blocks[(68,301)][0] 
b321 = blocks[(68,321)][0]
b331 = blocks[(68,331)][0]
b341 = blocks[(68,341)][0]

In [ ]:
b301.start_mega, b301.stop_mega

(108, 137)

In [ ]:
b321.start_mega, b321.stop_mega

(320, 340)

In [ ]:
b331.start_mega, b331.stop_mega

(400, 410)

In [ ]:
b341.start_mega, b341.stop_mega

(960, 1219)

Vemos portanto que o quê diferencia diferentes blocos do mesmo tipo mas com diferentes `thread_id` é a faixa de frequência de varredura. 

O primeiro tipo de bloco mostrado, o bloco do tipo `68` e thread_id `301`, é o mais numeroso do arquivo com 133932 blocos. 

A faixa desse arquivo é muito importante e de grande enfoque nas monitorações da Anatel - 108MHz a 137MHz. Essa faixa é a do Serviço Limitado Móvel Aeronáutico, dedicada a comunicações entre aeronaves com as torres de comando e entre si.

Cada bloco é uma medição num intervalo de tempo específico de diversos pontos nessa faixa de 108MHz a 137MHz. Portanto temos `133932` medições

In [ ]:
b301.data[b301.start:b301.stop]

b'\xff\xfaF\xff\xfaH\xff\xfaH\xff\xfaE\xff\xfaA\xff\xfaB\xff\xfaA\xff\xfa>\xff\xfa>\xff\xfa=\xff\xfa<\xff\xfa;\xff\xfa;\xff\xfa;\xff\xfa;\xff\xfa;\xff\xfa5\xff\xfa5\xff\xfa6\xff\xfa7\xff\xfa8\xff\xfa7\xff\xfa6\xff\xfa4\xffv`d`\xff\xfa6\xff\x9bamm`\xff\xfa6\xff\xfa8\xff\xfa8\xff\xfa8\xff\xfa8\xff\xfa7\xff\xfa6\xff\xfa5\xff\x8ba\xff\xfa4\xff\xfa6\xff\xfa4\xff\xfa5\xff\xfa5\xff\x97g|\x82{f\xff\xfa5\xff\xfa5\xff\xfa7\xff\xfa7\xff\xfa8\xff\xfa6\xff\xfa6\xff\xfa6\xff\xfa8\xff\xfa7\xff\xfa8\xff\xfa8\xff\xfa9\xff\xfa<\xff\xfa>\xff\xfa>\xff\xfaA\xff\xfaAB'

O Atributo `data` são os bytes brutos, i.e. não decodificados mas somente lidos. Os atributos `start` e `stop` recortam os bytes nos pontos correspondente às medidas de nível do arquivo binário, ignorando os demais pontos do arquivo que constituem metadados. O restante desse artigo é dedicado a decodificação dessas medidas de nível.

## Codificação Espectral
Dada estação faz uma medição de potência, para dada frequência, em dBm ( lê-se 'de-bê-eme' ), uma escala logarítmica com valores tipicamente negativos. A escala de valores que a estação armazena é a seguinte `Intervalo = [offset - 127.5, offset]`, onde `offset` (*deslocamento*) é um valor pré-estabelecido. Se a medida estiver fora desse intervalo, os valores são truncados. Um típico valor de `offset` é -*20dBm*

Assim tendo o valor medido `d` em dBm ele é codificado, da seguinte maneira:

$$b = 2(d - offset) + 255$$

Ao inserir os valores extremos do intervalo acima nessa fórmula:

Para `d = offset - 127.5`:

$$ \therefore b = 2[(offset - 127.5) - offset] + 255 = 2 (-127.5) + 255 \implies b = 0$$

Para `d = offset`:
$$\therefore b = 2(offset - offset) + 255 = 2 * 0 + 255 \implies b = 255$$

Portanto o intervalo de valores possíveis ao serem codificados com a fórmula acima é `[0,255]`, justamente o intervalo de valores possíveis em 8 bits ou 1 byte, sem sinal. Assim, os dados de espectro truncados e codificados dessa maneira permitem um armazenamento extremamente econômico, ocupando somente 1 byte (`uint8` em python ou `unsigned char` na linguagem C.)

Para decodificar tal valor, basta fazer o procedimento inverso  da fórmula acima. Ao isolar o valor `d` temos:
$$d = \frac{b}{2} + offset - 127.5$$

## Compressão de Dados
Como dados de espectro são extremamente ruidosos ( índice sinal-ruído muito baixo ), isto é, a maioria do que observamos numa dada "janela" do espectro é simplesmente ruído. A aplicação `Logger` comprime esses dados de maneira engenhosa. Como mencionado, a cada instante de tempo a estação faz a medição dos níveis de determinada faixa, para tal ela divide a faixa em diversos intervalos e mede o nível da frequência central daquele intervalo. Quanto maior o número de intervalos mais granulosa ou detalhada serão essas medidas.

**Algoritmo de Compressão**

* No script é definido um limiar `threshold`, abaixo do qual tudo é considerado ruído
* Para cada intervalo medido `i`, é verificado se o valor está abaixo do limiar
* Caso afirmativo esse processo se repete para os intervalos vizinhos `i+1, i+2, etc...`
* Esses intervalos são contados, até que apareça um ponto acima do limiar ou o máximo de `250` pontos seja atingido
  * Nesse ponto é gravado no arquivo um byte marcador `RUN=255` indicando que o próximo byte armazena a contagem de pontos abaixo do limiar
  * Nesse procedimento é ocupado somente 2 bytes, podendo ter sido comprimido o limite de 250 bytes no melhor dos casos.

Um problema que pode ocorrer no algoritmo acima é se nos dados de medição tivermos o valor máximo medido `d=offset` assim o valor a ser gravado será `b=255`, o mesmo valor usado como marcador. 

Nesses casos temos o valor literal no ponto e não queremos que isso seja interpretado como marcador. 

Para lidar com esse caso é definido um segundo marcador `ESC=254`, esse marcador informa o algoritmo de decodificação que a medida é literal e não se trata de um marcador.

Com alguns exemplos a seguir ficará mais claro o funcionamento do algoritmo. O algoritmo original de codificação ( escrito em C ) é apresentado a seguir:

```c
int run_length_encode(unsigned char *dest, unsigned char *src, int nsrc, int thresh) { 
    unsigned char ib; 
    int di = 0, si, nunder = 0; 
    for(si = 0; si < nsrc; si++){ 
        ib = src[si]; 
        if ((ib < thresh) && (si < (nsrc-1)) && (nunder < 250)) { 
            nunder += 1; 
        } else { 
            if (nunder > 0) { 
                dest[di++] = RUN; 
                dest[di++] = nunder; 
                nunder = 0; 
            } if ((ib == RUN) || (ib == ESC)) {
                dest[di++] = ESC; 
                dest[di++] = ib; 
            } else { 
                dest[di++] = ib; 
            } 
        } 
    } return di; 
}
```

O algoritmo é bastante legível mesmo para quem não conhece C, exceto talvez por algumas coisas estranhas como ponteiros `*` e a incrementação de variáveis dentro dos arrays. 
Vamos escrevê-lo em python tentando manter o estilo em C e comentando para decifrarmos melhor seu propósito. 

In [ ]:
RUN = 255
ESC = 254
def run_length_encode(destino, origem, n_origem, limiar):
    
    conta_origem, conta_destino = 0,0 # si, and di
    
    num_under = 0
    for si in range(n_origem): # percorro as posições do arquivo de origem
        
        byte_atual = origem[si] # leio o byte do arquivo fonte correspondente à posição si
        
        # Se o byte está abaixo do limiar e não atingi o fim do arquivo ou a contagem máxima
        if (byte_atual < limiar) and (si < n_origem - 1) and (num_under < 250):
            
            num_under += 1 # incremento o contador de valor abaixo do limiar
        else:
            if num_under > 0: # valor atual está acima do limiar mas até o momento estávamos contando valores abaixo do limiar
                
                destination[conta_destino] = RUN # insiro o marcador de contagem de valores abaixo do limiar na posição atual do arquivo de destino
                
                conta_destino += 1 # incremento o contador do arquivo de destino
                
                destination[conta_destino] = num_under # coloco a contagem em si na próxima posição
                
                conta_destino += 1 # incremento o contador do arquivo de destino
                
                num_under = 0 # zero o contador de valores abaixo do limiar
                
            if (byte_atual == RUN) or (byte_atual == ESC): # checo se o valor lido atual corresponde a um dos valores utilizados como marcador
                
                destination[conta_destino] = ESC # coloco o marcador que o valor a seguir lido é literal
                
                conta_destino += 1 # incremento o contador do arquivo de destino
                
                destination[conta_destino] = byte_atual
                
                conta_destino += 1 # incremento o contador do arquivo de destino
                
            else: # caso o valor medido não corresponda a um dos valores reservador de marcador eu simplesmente guardo esse valor na posição atual
                
                destination[conta_destino] = byte_atual
                
                conta_destino += 1 # incremento o contador do arquivo de destino
                
    return destination            

O algoritmo está propositalmente "não-pythônico", mais ao estilo de C para ficar o mais próximo do original e espaçado por conta dos vários comentários.  

Como sempre as coisas ficam mais claras com um exemplo

**Exemplo 1 - Somente valores abaixo do limiar, i.e. somente ruído**

In [ ]:
destination = [0] * 10 #lista com 10 valores 0
offset = -20
limiar = -80
medidas = [-100] * 10

Como vimos acima esses valores de medida antes de serem codificados são transformados com seguinte fórmula: $b = 2(d - offset) + 255$. Portanto nosso arquivo de origem fica:

In [ ]:
origem = [int(2 * (d - offset) + 255) for d in medidas] ; origem

[95, 95, 95, 95, 95, 95, 95, 95, 95, 95]

In [ ]:
limiar = 2 * (limiar - offset) + 255 

In [ ]:
run_length_encode(destination, origem, len(origem), limiar)

[255, 9, 95, 0, 0, 0, 0, 0, 0, 0]

Temos 10 valores, todos abaixo do limiar, então o algoritmo armazena o marcador `RUN=255` mais a contagem de valores a seguir. Era esperado que a contagem fosse 10 e não 9. No entanto pela implementação do algoritmo o último valor é armazenado literalmente, mesmo estando abaixo do limiar. Esse é um caso limite no qual todos os valores do espectro estão abaixo do limiar. Perceba que nesse caso limite 3 bytes foram ocupados dos 10 originais

**Exemplo 2 - Ruído mais valores normais**

In [ ]:
destination = [0] * 10 #lista com 10 valores 0
offset = -20
limiar = -80
medidas = [-100, -100, -100, -100, -100, -75, -62, -22, -30, -78]
origem = [int(2 * (d - offset) + 255) for d in medidas]
limiar = 2 * (limiar - offset) + 255 
origem

[95, 95, 95, 95, 95, 145, 171, 251, 235, 139]

In [ ]:
run_length_encode(destination, origem, len(origem), limiar)

[255, 5, 145, 171, 251, 235, 139, 0, 0, 0]

Temos 5 valores abaixo do limiar então é armazenado o marcador `RUN=255` seguido da contagem `[255,5...] e em seguida são armazenados os valores literais que estão acima do limiar.

**Exemplo 3 - Ruído, Valores Normais e valores extremos igual ao `offset`**

In [ ]:
destination = [0] * 10 #lista com 10 valores 0
offset = -20
limiar = -80
medidas = [-100, -100, -100, -100, -100, -20, -20.5, -22, -30, -78]
origem = [int(2 * (d - offset) + 255) for d in medidas]
limiar = 2 * (limiar - offset) + 255 
origem

[95, 95, 95, 95, 95, 255, 254, 251, 235, 139]

In [ ]:
run_length_encode(destination, origem, len(origem), limiar)

[255, 5, 254, 255, 254, 254, 251, 235, 139, 0]

Os valores medidos igual ou próximos ao offset `-20, -20.5` ao serem transformados viram `255 e 254`, os valores utilizados como marcador. Nesse caso teremos duas sinalizações de valor literal com o marcador `ESC=254`. 
* `[255,5...]` temos 5 valores abaixo do limiar
* `[...254,255...]` temos o valor literal `255`
* `[...254,254...]` temos o valor literal `254`
* `[...251,235,139]` valores de medição literal

Nesse caso economizamos somente 1 byte, denotado pelo valor 0 ao final da lista. Esses exemplos com poucos valores não fazem jus ao algoritmo, o poder de compressão aparece quando temos milhares de dados como veremos a seguir.

**Algoritmo de Descompressão**

O Algoritmo de descompressão original em C é mostrado a seguir:

```c
#define RUN 255 
#define ESC 254 
int run_length_decode(unsigned char *dest, unsigned char *src, int nsrc, int thresh) { 
    int si=0,di=0,nrun; 
    unsigned char ib; 
    while (si < nsrc) { 
        ib = src[si++]; 
        if (ib == RUN) { 
            nrun = src[si++];
            while(nrun-- >0){ 
                dest[di++] = thresh; }
        } else if (ib == ESC) {
            /* next value is literal */ 
            dest[di++] = src[si++];
        } else {
            /* value */ 
            dest[di++] = ib; 
        } 
    } return di; 
}
```

Vamos escrevê-lo em python.

In [ ]:
def run_length_decode(src, nsrc, thresh, offset):
    dest = [] # creates an empty destination list
    si = 0 #counter source
    di = 0 #counter destination 
    while si < nsrc: #while we didn't read the whole file
        ib = src[si] # read current position
        si+=1 # counter go to the next position
        if ib == RUN: # if current position is equal to marker RUN
            nrun = src[si] # next position indicates number of points below thresh
            si+=1
            while nrun > 0:
                dest.append(thresh) # we keep thresh in the destination nrun times
                di+=1
                nrun-=1
        elif ib == ESC: # next value is literal
            dest.append(src[di]/2 + offset - 127.5)
            di+=1 ; si+=1
        else:
            # value
            dest.append(ib/2 + offset - 127.5) # If there isn't a marker I'll just keep the current value
            di+=1
    return dest

O algoritmo basicamente faz:
* Percorre o arquivo codificado fonte e lê byte a byte
* Se o byte é igual ao marcador `RUN=255` é sabido que o byte seguinte armazena a contagem de quantas vezes foi medido um valor abaixo do limiar
* Um loop com essa contagem é efetuado e a cada rodada é armazenado o valor do limiar na lista de destino
* Caso seja identificado o outro marcador `ESC=254` é armazenado o valor seguinte
* Caso nenhum dos casos anteriores ocorra simplesmente é armazenado o valor atual

## Teste de Velocidade do Algoritmo Original
A motivação para a otimização da leitura desses arquivos comprimidos foi por conta de arquivos da ordem de 100MB, que ao serem descomprimidos geram por volta de 8GB de dados. 
Utilizamos um desses arquivos como exemplo.

Eu li e ouvi de mais de uma vez que não se faz otimização sem "profiling", isto é, não comece a otimizar as coisas antes de saber exatamente o quê  toma o tempo do seu código. 

Para nos ajudar nisso vamos usar a extensão da biblioteca `line_profiler` que mostra a execução linha a linha de dada função que passarmos como argumento.

In [ ]:
compressed_blocks = blocks[(68,301)]

In [ ]:
RUN = 255
ESC = 254    

In [ ]:
def size_in_gb(obj):
    print(f'{sys.getsizeof(obj)/1e6:.2f} GB')

A função a seguir testa o algoritmo original acima.

In [ ]:
#slow
def test_orig(blocks, debug=False):
    decoded = []
    if debug:
        blocks = blocks[:1]
    for block in tqdm(blocks):
        src = block.data[block.start:block.stop]
        nsrc = len(src) #block.stop - block.start
        thresh = block.thresh       
        offset = block.offset
        decoded.append(run_length_decode(src, nsrc, thresh, offset))
    return decoded

Vamos checar o perfil de uma chamada da função `run_length_decode`, isso é alcançado chamando a função acima com o argumento `debug=True`. 
Assim executamos somente 1 bloco em vez de 133932.

In [ ]:
%lprun -f run_length_decode test_orig(compressed_blocks, debug=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Timer unit: 3.52617e-07 s

Total time: 0.0316763 s
File: <ipython-input-19-0e35b2a102ae>
Function: run_length_decode at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_length_decode(src, nsrc, thresh, offset):
     2         1          4.0      4.0      0.0      dest = [] # creates an empty destination list
     3         1          2.0      2.0      0.0      si = 0 #counter source
     4         1          1.0      1.0      0.0      di = 0 #counter destination 
     5       131        188.0      1.4      0.2      while si < nsrc: #while we didn't read the whole file
     6       130        230.0      1.8      0.3          ib = src[si] # read current position
     7       130        181.0      1.4      0.2          si+=1 # counter go to the next position
     8       130        225.0      1.7      0.3          if ib == RUN: # if current position is equal to marker RUN
     9        60         84.0      1.4 

Vemos que 98.3% do tempo a função passa somente no loop `while` interno. Vamos ver o que isso significa para 133932 blocos.

In [ ]:
%%time
d = test_orig(compressed_blocks)

  0%|          | 0/133932 [00:00<?, ?it/s]

Wall time: 4min 39s


Isso no contexto de uso do dia a dia do fiscal da Anatel é um tempo extenso, porque pode haver vários arquivos e essas análises podem ser recorrentes.

In [ ]:
len(d), len(d[0])

(133932, 14633)

In [ ]:
size_in_gb(d)

1.17 GB


In [ ]:
del d
gc.collect()

66

### Loop `while` Eliminado
Vemos que o loop while acima simplesmente adiciona o mesmo valor `thresh` por `nrun` vezes. Para eliminarmos o loop basta extendermos a lista de uma só vez, adicionando a ela uma outra lista de tamanho `nrun` povoada com valores `thresh`.

In [ ]:
def run_length_decode2(dest, src, nsrc, thresh, offset):
    si = 0 #counter source
    di = 0 #counter destination 
    while si < nsrc: #while we didn't read the whole file
        ib = src[si] # read current position
        si+=1 # counter go to the next position
        if ib == RUN: # if current position is equal to marker RUN
            nrun = src[si] # next position indicates number of points below thresh
            si+=1
            di+=nrun # Full incremental
            dest.extend([thresh]*nrun) #Extend the resulted list in a pythonic way 
        elif ib == ESC: # next value is literal
            dest.append(src[di]/2 + offset - 127.5)
            di+=1 ; si+=1
        else:
            # value
            dest.append(ib/2 + offset - 127.5) # If there isn't a marker I'll just keep the current value
            di+=1
    return dest

Substituímos:

```python
while nrun > 0:
    dest.append(thresh)
    di+=1
    nrun-=1
```
por
```python
di+=nrun
dest.extend([thresh]*nrun)
```

In [ ]:
def test_no_while(blocks, debug=False):
    decoded = []
    if debug:
        blocks= blocks[:1]
    for block in tqdm(blocks):
        dest = []
        src = block.data[block.start:block.stop]
        nsrc = len(src) #block.stop - block.start
        thresh = block.thresh       
        offset = block.offset
        decoded.append(run_length_decode2(dest, src, nsrc, thresh, offset))
    return decoded

In [ ]:
%lprun -f run_length_decode2 test_no_while(compressed_blocks, debug=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Timer unit: 3.52617e-07 s

Total time: 0.00457802 s
File: <ipython-input-29-d484ae5f5462>
Function: run_length_decode2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_length_decode2(dest, src, nsrc, thresh, offset):
     2         1          5.0      5.0      0.0      si = 0 #counter source
     3         1        604.0    604.0      4.7      di = 0 #counter destination 
     4       131        207.0      1.6      1.6      while si < nsrc: #while we didn't read the whole file
     5       130        318.0      2.4      2.4          ib = src[si] # read current position
     6       130        145.0      1.1      1.1          si+=1 # counter go to the next position
     7       130        204.0      1.6      1.6          if ib == RUN: # if current position is equal to marker RUN
     8        60        114.0      1.9      0.9              nrun = src[si] # next position indicates number of points below thre

O tempo caiu de `0.0316763s` para `0.00457802s`, ainda passamos a maior parte do tempo, `82.8%`, extendendo a lista original, no entanto isso é feito uma única vez por byte.

Vamos ver como isso se traduz ao executarmos a função para todos os blocos.

In [ ]:
%%time
d = test_no_while(compressed_blocks)

  0%|          | 0/133932 [00:00<?, ?it/s]

Wall time: 1min 12s


Passamos de 4min e 39s para 1min e 12s. Melhoria de `279s / 72s = 3,875 vezes`

In [ ]:
len(d), len(d[0])

(133932, 14633)

In [ ]:
size_in_gb(d)

1.17 GB


In [ ]:
del d
gc.collect()

66

### Trocar de Listas para arrays pré-alocados
Como a maioria do tempo dispendido na função anterior ainda é na extensão da lista original com a lista de valores threshold. 
Para cada bloco a lista é extendida com uma sublista povoada com os valores de limiar `thresh`. Isso demanda que a memória seja alocada toda vez que extendemos a lista. 

Nessa alocação os endereços de memória não necessariamente estão adjacentes, isso certamente gera um gargalo no tempo de execução. 

Esse problema também aparece na função externa `test_no_while`.

Para sanarmos isso vamos declarar um numpy array já com os valores de `thresh` preenchidos, assim realizamos uma pré-alocação de memória eficiente ( valores adjacentes ) e para os valores lidos abaixo do limiar não precisamos fazer nenhuma atribuição visto que o numpy array já está povoado com valores `thresh`. 

In [ ]:
def run_length_decode3(dest, src, nsrc, thresh, offset):
    i = 0
    j = 0
    while i < nsrc:
        ib = src[i] 
        i+=1
        if ib == RUN:
            nrun = src[i] 
            i+=1
            dest[j:j+nrun] = thresh #dest is now a numpy array
            j+=nrun
        elif ib == ESC:
            # next value is literal
            dest[j] = src[i]/2. + offset - 127.5
            i+=1 ; j+=1
        else:
            # value
            dest[j] = ib/2. + offset - 127.5
            j+=1    

In [ ]:
#slow
def test_prealloc_np(blocks):
    decoded = np.empty((len(blocks), blocks[0].norig), dtype=np.float16)
    decoded.fill(blocks[0].thresh)
    for b, block in enumerate(tqdm(blocks)):
        src = block.data[block.start:block.stop]
        nsrc = len(src)
        thresh = block.thresh
        offset = block.offset
        dest = np.empty(block.norig, dtype=np.float16)
        dest.fill(thresh)
        decoded[b] = run_length_decode3(dest, src, nsrc, thresh, offset)
    return decoded

Também declaramos os dados como sendo do tipo, `np.float16`, dado que os valores de nível do espectro possuem precisão de no máximo 1 casa decimal, o valor`np.float16` é o suficiente para armazená-los

In [ ]:
%lprun -f run_length_decode3 test_prealloc_np(compressed_blocks[:1])

  0%|          | 0/1 [00:00<?, ?it/s]

Timer unit: 3.52617e-07 s

Total time: 0.000689366 s
File: <ipython-input-23-be56a10ab4a1>
Function: run_length_decode3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_length_decode3(dest, src, nsrc, thresh, offset):
     2         1          5.0      5.0      0.3      i = 0
     3         1          2.0      2.0      0.1      j = 0
     4       131        167.0      1.3      8.5      while i < nsrc:
     5       130        257.0      2.0     13.1          ib = src[i] 
     6       130        192.0      1.5      9.8          i+=1
     7       130        201.0      1.5     10.3          if ib == RUN:
     8        60        118.0      2.0      6.0              nrun = src[i] 
     9        60        163.0      2.7      8.3              i+=1
    10        60        225.0      3.8     11.5              dest[j:j+nrun] = thresh #dest is now a numpy array
    11        60        116.0      1.9      5.9         

Usando um array pré-alocado o tempo do algoritmo para decodificar um bloco caiu de `0.00457802s` para `0.000689366s`. O efeito mais significativo será na decodificação de todos os blocos, onde também é usada pré-alocação. 

In [ ]:
%%time
d = test_prealloc_np(compressed_blocks)

  0%|          | 0/133932 [00:00<?, ?it/s]

Wall time: 29.5 s


Temos uma queda de `1min e 12s` para `29.5s`

In [ ]:
del d
gc.collect()

139

Algumas melhorias mais sutis podem ser feitas para reduzir o número de operações redundantes efetuadas.
* O valor de `thresh` e `offset` é determinado no script, então é o mesmo para todos os blocos. 
Então não precisamos extraí-los toda vez, basta armazená-los uma única vez e repassá-los para a função.
* A operação `+ offset - 127.5` constitui uma soma e subtração de valores fixos então pode ser reduzida a somente 1 operação com valor fixo: `MIN = offset - 127.5`

In [ ]:
def run_length_decode4(dest, src, nsrc, thresh, MIN):
    i = 0
    j = 0
    while i < nsrc:
        ib = src[i] 
        i+=1
        if ib == RUN:
            nrun = src[i] 
            i+=1
            dest[j:j+nrun] = thresh #dest is now a numpy array
            j+=nrun
        elif ib == ESC:
            # next value is literal
            dest[j] = src[i]/2. + MIN
            i+=1 ; j+=1
        else:
            # value
            dest[j] = ib/2. + MIN
            j+=1  
    return dest

In [ ]:
#slow
def test_prealloc_np2(blocks):
    decoded = np.empty((len(blocks), blocks[0].norig), dtype=np.float16)
    decoded.fill(blocks[0].thresh)
    thresh = blocks[0].thresh
    offset = blocks[0].offset 
    MIN = offset - 127.5
    for b, block in enumerate(tqdm(blocks)):
        src = block.data[block.start:block.stop]
        nsrc = len(src)
        dest = np.empty(block.norig, dtype=np.float16)
        dest.fill(thresh)
        decoded[b] = run_length_decode4(dest, src, nsrc, thresh, MIN)
    return decoded

In [ ]:
%lprun -f run_length_decode4 test_prealloc_np2(compressed_blocks[:1])

  0%|          | 0/1 [00:00<?, ?it/s]

Timer unit: 3.52617e-07 s

Total time: 0.000672088 s
File: <ipython-input-26-bf6ce6499759>
Function: run_length_decode4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_length_decode4(dest, src, nsrc, thresh, MIN):
     2         1          5.0      5.0      0.3      i = 0
     3         1          2.0      2.0      0.1      j = 0
     4       131        182.0      1.4      9.5      while i < nsrc:
     5       130        264.0      2.0     13.9          ib = src[i] 
     6       130        214.0      1.6     11.2          i+=1
     7       130        235.0      1.8     12.3          if ib == RUN:
     8        60         68.0      1.1      3.6              nrun = src[i] 
     9        60        100.0      1.7      5.2              i+=1
    10        60        283.0      4.7     14.8              dest[j:j+nrun] = thresh #dest is now a numpy array
    11        60         67.0      1.1      3.5            

O efeito para 1 bloco somente é muito pequeno, vamos ver o efeito na decodificação de todos os blocos.

In [ ]:
d = test_prealloc_np2(compressed_blocks)

  0%|          | 0/133932 [00:00<?, ?it/s]

Diminuímos de 29.5s para 23s

In [ ]:
del d
gc.collect()

53

In [ ]:
def run_length_decode5(dest, src, nsrc, thresh, MIN):
    i = 0
    j = 0
    while i < nsrc:
        ib = src[i] 
        i+=1
        if ib == RUN:
            nrun = src[i] 
            i+=1
            dest[j:j+nrun] = thresh #dest is now a numpy array
            j+=nrun
        elif ib == ESC:
            # next value is literal
            dest[j] = src[i]/2. + MIN
            i+=1 ; j+=1
        else:
            # value
            dest[j] = ib/2. + MIN
            j+=1  

In [ ]:
#slow
def test_prealloc_np3(blocks):
    thresh = blocks[0].thresh
    decoded = np.full((len(blocks), blocks[0].norig), thresh, dtype=np.float16)
    offset = blocks[0].offset 
    MIN = offset - 127.5
    for b, block in enumerate(tqdm(blocks)):
        src = block.data[block.start:block.stop]
        nsrc = len(src)
        #dest = np.empty(block.norig, dtype=np.float16)
        #dest.fill(thresh)
        run_length_decode5(decoded[b, :], src, nsrc, thresh, MIN)
    return decoded

In [ ]:
%lprun -f run_length_decode5 test_prealloc_np3(compressed_blocks[:1])

Timer unit: 3.52617e-07 s

Total time: 0.000476738 s
File: <ipython-input-37-5c3575d407da>
Function: run_length_decode5 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def run_length_decode5(dest, src, nsrc, thresh, MIN):
     2         1         41.0     41.0      3.0      i = 0
     3         1          3.0      3.0      0.2      j = 0
     4       131        137.0      1.0     10.1      while i < nsrc:
     5       130        141.0      1.1     10.4          ib = src[i] 
     6       130        135.0      1.0     10.0          i+=1
     7       130        140.0      1.1     10.4          if ib == RUN:
     8        60         65.0      1.1      4.8              nrun = src[i] 
     9        60         62.0      1.0      4.6              i+=1
    10        60        230.0      3.8     17.0              dest[j:j+nrun] = thresh #dest is now a numpy array
    11        60         70.0      1.2      5.2            

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
d = test_prealloc_np3(compressed_blocks)

  0%|          | 0/133932 [00:00<?, ?it/s]

In [ ]:
del d
gc.collect()

204

### Processamento Paralelo dos Blocos
Como estamos iterando os vários blocos e aplicando a função de decodificação, outra otimização que vêm em mente é paralelizar essa chamada de função.

Para isso precisamos adaptar a função para podermos dividir não somente o bloco como o array de destino

In [ ]:
#slow
from rfpy.parser import run_length_decode5
def test_prealloc_mp(blocks):
    thresh = blocks[0].thresh
    decoded = np.full((len(blocks), blocks[0].norig), thresh, dtype=np.float16)
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(run_length_decode5, decoded)
    return decoded

In [ ]:
%%time
d = test_prealloc_mp(compressed_blocks)

TypeError: run_length_decode5() missing 4 required positional arguments: 'src', 'nsrc', 'thresh', and 'MIN'

In [ ]:
#slow
def test_prealloc_fill(blocks):
    decoded = np.full((len(blocks), blocks[0].norig), MIN, dtype=np.float16)

    for b, block in enumerate(tqdm(blocks)):
        src = block.data[block.start:block.stop]
        nsrc = len(src)
        thresh = block.thresh
        dest = np.full(block.norig, MIN, dtype=np.float16)
        i = 0
        j = 0
        while i < nsrc:
            ib = src[i] 
            i+=1
            if ib == RUN:
                nrun = src[i] 
                i+=1
                dest[j:j+nrun] = MIN + thresh/2.
                j+=nrun
            elif ib == ESC:
                # next value is literal
                dest[j] = MIN + src[i]/2.
                i+=1 ; j+=1
            else:
                # value
                dest[j] = MIN + ib/2.
                j+=1
        decoded[b] = dest
    return decoded

## Functional Programming - Much Worse
Elimination of loops doesn't necessarily really eliminates them. `filter` and `map` run along the array

In [ ]:
#slow
def test_prealloc_functional(blocks):
    decoded = np.full((len(blocks), blocks[0].norig), MIN, dtype=np.float16)
    MAX = blocks[0].norig
    for b, block in enumerate(tqdm(blocks)):
        src = block.data[block.start:block.stop]
        threshold = block.thresh
        dest = np.concatenate(L(src.split(b'\xff')).filter(lambda o: o != b'').map(lambda o: np.concatenate([np.repeat(threshold,o[0]),  np.fromiter(o[1:].replace(b'\xfe', b''), dtype=np.float16, count=len(o[1:]))])))
        decoded[b][:dest.shape[0]] = dest[:MAX]
    return decoded

In [ ]:
# %%time
# decoded = test_prealloc_mp(compressed_blocks)

In [ ]:
#%load_ext Cython

In [ ]:
# %%cython

# cimport cython

# import numpy as np
# cimport numpy as np
# #from cython.parallel import prange

# #ctypedef np.double_t DTYPE_t

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef object cy_decode_block(block):
#     cdef float MIN = block.offset - 127.5
#     cdef int RUN = 255
#     cdef int ESC = 254
#     cdef const unsigned char[:] src = block.data[block.start:block.stop]
#     src = block.data[block.start:block.stop]
#     cdef int nsrc = len(src)
#     cdef int thresh = block.thresh
#     cdef np.ndarray dest =  np.full(block.norig, MIN, dtype=np.float16)
#     cdef int i = 0
#     cdef int j = 0
#     while i < nsrc:
#         ib = src[i] 
#         i+=1
#         if ib == RUN:
#             nrun = src[i] 
#             i+=1
#             dest[j:j+nrun] = MIN + thresh/2.
#             j+=nrun
#         elif ib == ESC:
#             # next value is literal
#             dest[j] = MIN + src[i]/2.
#             i+=1 ; j+=1
#         else:
#             # value
#             dest[j] = MIN + ib/2.
#             j+=1
#     return dest

In [ ]:
# %%cython --annotate

# cimport cython

# import numpy as np
# cimport numpy as np
# #from cython.parallel import prange

# #ctypedef np.double_t DTYPE_t

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef object cy_decode_blocks(list blocks):
#     cdef float offset = blocks[0].offset
#     cdef float MIN = offset - 127.5
#     cdef int rows = len(blocks)
#     cdef int columns = blocks[0].norig
#     cdef np.ndarray decoded = np.full((rows, columns), MIN, dtype=np.float16)
#     cdef int RUN = 255
#     cdef int ESC = 254
#     cdef object block
#     cdef int row
#     cdef const unsigned char[:] src
#     cdef int nsrc
#     cdef int thresh = blocks[0].thresh
#     #cdef np.ndarray dest
#     cdef int i
#     cdef int j
#     for row, block in enumerate(blocks):
#         src = block.data[block.start:block.stop]
#         nsrc = len(src)
#         i = 0
#         j = 0
#         while i < nsrc:
#             ib = src[i] 
#             i+=1
#             if ib == RUN:
#                 nrun = src[i] 
#                 i+=1
#                 decoded[row, j:j+nrun] = MIN + thresh/2.
#                 j+=nrun
#             elif ib == ESC:
#                 # next value is literal
#                 decoded[row, j] = MIN + src[i]/2.
#                 i+=1 ; j+=1
#             else:
#                 # value
#                 decoded[row, j] = MIN + ib/2.
#                 j+=1
#     return decoded

In [ ]:
# %%cython --annotate

# cimport cython

# import numpy as np
# cimport numpy as np
# #from cython.parallel import prange

# #ctypedef np.double_t DTYPE_t

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef object cy_decode_blocks(list blocks):
#     cdef float offset = blocks[0].offset
#     cdef float MIN = offset - 127.5
#     cdef int rows = len(blocks)
#     cdef int columns = blocks[0].norig
#     cdef np.ndarray decoded = np.full((rows, columns), MIN, dtype=np.float16)
#     cdef list data = [block.data[block.start:block.stop] for block in blocks]
#     cdef int RUN = 255
#     cdef int ESC = 254
#     cdef int row
#     cdef const unsigned char[:] src
#     cdef int nsrc
#     cdef int thresh = blocks[0].thresh
#     cdef int i
#     cdef int j
#     for row in range(rows):
#         src = data[row]
#         nsrc = len(src)
#         i = 0
#         j = 0
#         while i < nsrc:
#             ib = src[i] 
#             i+=1
#             if ib == RUN:
#                 nrun = src[i] 
#                 i+=1
#                 decoded[row, j:j+nrun] = MIN + thresh/2.
#                 j+=nrun
#             elif ib == ESC:
#                 # next value is literal
#                 decoded[row, j] = MIN + src[i]/2.
#                 i+=1 ; j+=1
#             else:
#                 # value
#                 decoded[row, j] = MIN + ib/2.
#                 j+=1
#     return decoded

In [ ]:
# %%cython --annotate

# cimport cython

# import numpy as np
# cimport numpy as np
# #from cython.parallel import prange

# #ctypedef np.float_t DTYPE_t

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef object cy_decode_blocks(list blocks):
#     cdef float offset = blocks[0].offset
#     cdef float MIN = offset - 127.5
#     cdef int rows = len(blocks)
#     cdef int columns = blocks[0].norig
#     cdef np.ndarray decoded = np.full((rows, columns), MIN, dtype=np.float16)
#     #cdef DTYPE_t [:, :] decoded_v = decoded
#     cdef list data = [b.data[b.start:b.stop] for b in blocks]
#     cdef int RUN = 255
#     cdef int ESC = 254
#     cdef const unsigned char[:] src
#     cdef int NRSC
#     cdef float thresh = blocks[0].thresh
#     cdef int i
#     cdef int j
#     cdef int ib
#     cdef int nrun
#     cdef Py_ssize_t row   
#     for row in range(rows):
#         src = data[row]
#         nsrc = len(src)
#         i = 0
#         j = 0
#         while i < nsrc:
#             ib = src[i] 
#             i+=1
#             if ib == RUN:
#                 nrun = src[i] 
#                 i+=1
#                 decoded[row, j:j+nrun] = MIN + thresh/2.
#                 j+=nrun
#             elif ib == ESC:
#                 # next value is literal
#                 decoded[row, j] = MIN + src[i]/2.
#                 i+=1 ; j+=1
#             else:
#                 # value
#                 decoded[row, j] = MIN + ib/2.
#                 j+=1
#     return decoded

In [ ]:
# %%cython --annotate

# cimport cython

# import numpy as np
# cimport numpy as np
# #from cython.parallel import prange

# #DTYPE = np.float32

# ctypedef np.float32_t DTYPE_t

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef object cy_decode_blocks(list blocks):
#     cdef float offset = blocks[0].offset
#     cdef float MIN = offset - 127.5
#     cdef int rows = len(blocks)
#     cdef int columns = blocks[0].norig
#     cdef np.ndarray[DTYPE_t, ndim=2] decoded = np.full((rows, columns), MIN, np.float32)
#     cdef list data = [b.data[b.start:b.stop] for b in blocks]
#     cdef int RUN = 255
#     cdef int ESC = 254
#     cdef const unsigned char[:] src
#     cdef int NRSC
#     cdef int thresh = blocks[0].thresh
#     cdef int i
#     cdef int j
#     cdef int ib
#     cdef int nrun
#     cdef Py_ssize_t row   
#     for row in range(rows):
#         src = data[row]
#         nsrc = len(src)
#         i = 0
#         j = 0
#         while i < nsrc:
#             ib = src[i]
#             i+=1
#             if ib == RUN:
#                 nrun = src[i] 
#                 i+=1
#                 #decoded[row, j:j+nrun] = thresh # MIN + thresh/2.
#                 for _ in range(nrun):
#                     decoded[row, j] = MIN + thresh / 2.
#                     #decoded[row, j] = thresh                    
#                     j+=1
#                 #j+=nrun
#             elif ib == ESC:
#                 # next value is literal
#                 decoded[row, j] = MIN + src[i]/2.
#                 i+=1 ; j+=1
#             else:
#                 # value
#                 decoded[row, j] = MIN + ib/2.
#                 j+=1
#     return decoded

In [ ]:
%%cython --annotate

cimport cython

import numpy as np
cimport numpy as np
#from cython.parallel import prange

#DTYPE = np.float32

ctypedef np.float32_t DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef np.ndarray[DTYPE_t, ndim=2] cy_decode_blocks(list data, int rows, int cols, int thresh, float MIN):
    cdef np.ndarray[DTYPE_t, ndim=2] decoded = np.full((rows, cols), thresh, np.float32)
    cdef const unsigned char[:] src
    cdef int RUN = 255
    cdef int ESC = 254
    cdef int NRSC
    cdef int i
    cdef int j
    cdef int ib
    cdef int nrun
    cdef Py_ssize_t row   
    for row in range(rows):
        src = data[row]
        nsrc = len(src)
        i = 0
        j = 0
        while i < nsrc:
            ib = src[i]
            i+=1
            if ib == RUN:
                nrun = src[i] 
                i+=1
                for _ in range(nrun):
                    decoded[row, j] = thresh
                    j+=1
            elif ib == ESC:
                # next value is literal
                decoded[row, j] = MIN + src[i]/2.
                i+=1 ; j+=1
            else:
                # value
                decoded[row, j] = MIN + ib/2.
                j+=1
    return decoded

In [ ]:
def test_cython(blocks):
    return cy_decode_blocks(list(blocks))

In [ ]:
def test_mp_cy(blocks):
    MIN = blocks[0].offset - 127.5
    decoded = np.full((len(blocks), blocks[0].norig), MIN, dtype=np.float16)
    block_array = [(a,b) for a,b in zip(decoded, blocks)]
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(decode_blocks, block_array)
    return decoded

In [ ]:
offset = compressed_blocks[0].offset
MIN = offset - 127.5
cols = compressed_blocks[0].norig
thresh = compressed_blocks[0].thresh - 1
rows = len(compressed_blocks)
data_blocks = [b.data[b.start:b.stop] for b in compressed_blocks]

In [ ]:
frequencies = np.linspace(108,137, num=14848)

In [ ]:
%%time
decoded = pd.DataFrame(cy_decode_blocks(data_blocks, rows, cols, thresh, MIN), columns=frequencies)

In [ ]:
# %%time
# decoded = cy_decode_blocks(data_blocks, rows, cols, thresh, MIN)

In [ ]:
display(decoded[:5, :5])
display(decoded[-5:, -5:])

In [ ]:
print(f'{sys.getsizeof(decoded)/1e9:.2f} GB')

In [ ]:
decoded.shape

In [ ]:
TH = -80
dict_freq = {}
for c in tqdm(decoded.columns):
    d = decoded.loc[:, c].value_counts().to_dict()
    d = {k:v for k,v in d.items() if k > TH}
    if d:
        dict_freq[c] = d

In [ ]:
dict_freq

In [ ]:
del decoded
gc.collect()

In [ ]:
%%time
np.asarray([b.wallclock_datetime for b in compressed_blocks]).shape